In [ ]:
import os
import subprocess
import sys

# We need to copy globals to env otherwise other modules won't see them.
for k, v in list(globals().items()):
    if k.isupper() and not callable(v) and v is not None:
        os.environ[k] = str(v)


def install_if_missing(package_spec: str, import_name=None):
    import_name = import_name or package_spec.split()[0]
    try:
        __import__(import_name)
    except ImportError:
        print(f"{import_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_spec])


install_if_missing(
    "wordlift-sdk @ git+https://github.com/wordlift/python-sdk.git@2.5.6",
    "wordlift_sdk",
)
install_if_missing(
    "python-dotenv@^1.1.0",
    "dotenv",
)
install_if_missing("beautifulsoup4", "bs4")

In [ ]:
import logging
from wordlift_sdk.configuration import ConfigurationProvider
from wordlift_sdk.container import ApplicationContainer

from dotenv import load_dotenv

# Set all existing root handlers to WARNING
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Reconfigure logging to show WARNING and above by default
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s  %(levelname)-8s  %(name)s: %(message)s",
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

load_dotenv()


async def main(filepath: str):
    configuration_provider = ConfigurationProvider.create(filepath)
    application_container = ApplicationContainer(configuration_provider)
    kg_import_workflow = await application_container.create_kg_import_workflow()
    await kg_import_workflow.run()


await main("config/default.py")